## Importing libraries

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

### Define Fourtquare credentials

In [3]:
CLIENT_ID = 'WCK535NG04YZTNLZP3JNTI215B45UCMROLCZAO2DCWL4ZUQ2' # your Foursquare ID
CLIENT_SECRET = '1UVSHMRNYHQBMSEOGAL4GP3CACE24GL3LQ40DFVWADWGOY3Q' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WCK535NG04YZTNLZP3JNTI215B45UCMROLCZAO2DCWL4ZUQ2
CLIENT_SECRET:1UVSHMRNYHQBMSEOGAL4GP3CACE24GL3LQ40DFVWADWGOY3Q


#### Let's start converting the Mexico's jewerly neigborhood location in to latitud and longitud

In [4]:
address = 'Polanco, Mexico DF'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

19.43353 -99.190915


#### Now, lets look for all the jewerlies that are located in that neighborhood

In [5]:
search_query = 'Jewelry'
radius = 1000
print(search_query + ' .... OK!')

Jewelry .... OK!


In [6]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=WCK535NG04YZTNLZP3JNTI215B45UCMROLCZAO2DCWL4ZUQ2&client_secret=1UVSHMRNYHQBMSEOGAL4GP3CACE24GL3LQ40DFVWADWGOY3Q&ll=19.43353,-99.190915&v=20180604&query=Jewelry&radius=1000&limit=30'

#### Now let's examine the results

In [7]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d5ea5f7c53093002c56726c'},
 'response': {'venues': [{'id': '55240db2498e4fee5d59611a',
    'name': 'Floripa Jewelry',
    'location': {'address': 'Pasaje Polanco Local 13 Av. Pdte Mazaryk 360 Col. Polanco',
     'crossStreet': 'Julio Verne',
     'lat': 19.431596755981445,
     'lng': -99.1963119506836,
     'labeledLatLngs': [{'label': 'display',
       'lat': 19.431596755981445,
       'lng': -99.1963119506836}],
     'distance': 606,
     'postalCode': '11560',
     'cc': 'MX',
     'city': 'Ciudad de México',
     'state': 'Distrito Federal',
     'country': 'México',
     'formattedAddress': ['Pasaje Polanco Local 13 Av. Pdte Mazaryk 360 Col. Polanco (Julio Verne)',
      '11560 Ciudad de México, Distrito Federal',
      'México']},
    'categories': [{'id': '4bf58dd8d48988d111951735',
      'name': 'Jewelry Store',
      'pluralName': 'Jewelry Stores',
      'shortName': 'Jewelry',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories

#### Get relevant part of JSON and transform it into a *pandas* dataframe (eventought it is just one) 

In [8]:
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d111951735', 'name': 'J...",False,55240db2498e4fee5d59611a,Pasaje Polanco Local 13 Av. Pdte Mazaryk 360 C...,MX,Ciudad de México,México,Julio Verne,606,[Pasaje Polanco Local 13 Av. Pdte Mazaryk 360 ...,"[{'label': 'display', 'lat': 19.43159675598144...",19.431597,-99.196312,11560,Distrito Federal,Floripa Jewelry,v-1566483959


##### It seems that the closest Jewerly in that Polanco's neighborhoos in the Floripa's jewelry.

#### Let's define the data frame with the information that is of interest

In [9]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Floripa Jewelry,Jewelry Store,Pasaje Polanco Local 13 Av. Pdte Mazaryk 360 C...,MX,Ciudad de México,México,Julio Verne,606,[Pasaje Polanco Local 13 Av. Pdte Mazaryk 360 ...,"[{'label': 'display', 'lat': 19.43159675598144...",19.431597,-99.196312,11560,Distrito Federal,55240db2498e4fee5d59611a


#### Now let's see what are other jewelries close to our location

In [10]:
dataframe_filtered.name

0    Floripa Jewelry
Name: name, dtype: object

In [11]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a green circle marker to represent the Floripa Jewelry
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='green',
    popup='Floripa Jewelry',
    fill = True,
    fill_color = 'green',
    fill_opacity = 0.5
).add_to(venues_map)

# add the jewelries as purplee circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='purple',
        popup=label,
        fill = True,
        fill_color='purple',
        fill_opacity=0.5
    ).add_to(venues_map)


venues_map

####  Let's explore the closest shopping centers

##### Centro Comercial Polanco

In [12]:
venue_id = '4fa2b148771687ceb6022502' # Centro comercial Polanco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4fa2b148771687ceb6022502?client_id=WCK535NG04YZTNLZP3JNTI215B45UCMROLCZAO2DCWL4ZUQ2&client_secret=1UVSHMRNYHQBMSEOGAL4GP3CACE24GL3LQ40DFVWADWGOY3Q&v=20180604'

In [13]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4fa2b148771687ceb6022502',
 'name': 'Centro Comercial Polanco',
 'contact': {},
 'location': {'address': 'Blvd. Miguel de Cervantes Saavedra 397',
  'crossStreet': 'Entre Presa Falcón y Presa Angostura',
  'lat': 19.44110762730647,
  'lng': -99.20659748338925,
  'labeledLatLngs': [{'label': 'display',
    'lat': 19.44110762730647,
    'lng': -99.20659748338925}],
  'postalCode': '11500',
  'cc': 'MX',
  'city': 'Miguel Hidalgo',
  'state': 'Distrito Federal',
  'country': 'México',
  'formattedAddress': ['Blvd. Miguel de Cervantes Saavedra 397 (Entre Presa Falcón y Presa Angostura)',
   '11500 Miguel Hidalgo, Distrito Federal',
   'México']},
 'canonicalUrl': 'https://foursquare.com/v/centro-comercial-polanco/4fa2b148771687ceb6022502',
 'categories': [{'id': '4bf58dd8d48988d1fd941735',
   'name': 'Shopping Mall',
   'pluralName': 'Shopping Malls',
   'shortName': 'Mall',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/mall_',
    'suffix': '.png'},
   'prima

#### Get the mall's overall rating

In [14]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has no ratinmgs.')

7.6


##### Let's see if we can find a place with a better rating

In [15]:
venue_id = '4b74b161f964a520c7ec2de3' # ID of Palacio del Hierro
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

9.2


##### That is a really good rating. We'll keep this one. Let's analyze it more in detail:

##### Lets get the number of tips it has on Foursquare

In [16]:
result['response']['venue']['tips']['count']

326

#### Let's get the venues tips
##### The idea would be to get all the tips but as we have a personal account it would not allow us...

In [17]:
limit = 2
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results


{'meta': {'code': 200, 'requestId': '5d5ea60da197930038cb4aea'},
 'response': {'tips': {'count': 326,
   'items': [{'id': '57ba7110498e347034f6e3da',
     'createdAt': 1471836432,
     'text': 'No existe Palacio que se le compare, es verdaderamente el Palacio de los Palacios y uno de mis lugares favoritos en la ciudad, desde la@planta baja hasta el último piso no hay más que cosas para gozar',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/57ba7110498e347034f6e3da',
     'photo': {'id': '57ba7117498e6b1fc265fec2',
      'createdAt': 1471836439,
      'source': {'name': 'Foursquare for iOS',
       'url': 'https://foursquare.com/download/#/iphone'},
      'prefix': 'https://fastly.4sqi.net/img/general/',
      'suffix': '/89714154_loqiGMGZ8Fp67jDpkOkzGk_OcuE7FJr9hDNHLLxToq8.jpg',
      'width': 1920,
      'height': 1440,
      'visibility': 'public'},
     'photourl': 'https://fastly.4sqi.net/img/general/original/89714154_loqiGMGZ8Fp67jDpkOkzGk_OcuE7FJr9hDNHLLxT

In [18]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'photo', 'photourl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'lastVoteText', 'lastUpvoteTimestamp', 'todo', 'user', 'authorInteractionType'])

#### Now, let's fix it in a  data frame

In [19]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) 

filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"No existe Palacio que se le compare, es verdaderamente el Palacio de los Palacios y uno de mis lugares favoritos en la ciudad, desde la@planta baja hasta el último piso no hay más que cosas para gozar",13,0,57ba7110498e347034f6e3da,Gibrán,Flores,male,89714154


### Next step is do the analysis for Bogota in order to compare the two neighborhoods.

In [20]:
address2 = 'Bogota'

geolocator2 = Nominatim(user_agent="foursquare_agent")
location2 = geolocator2.geocode(address2)
latitude2 = location2.latitude
longitude2 = location2.longitude
print(latitude2, longitude2)

4.59808 -74.0760439


#### Now, lets look for all the jewerlies that are located in the city

In [21]:
search_query2 = 'Joyeria'
radius = 1000
print(search_query + ' .... OK!')

Jewelry .... OK!


In [22]:
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude2, longitude2, VERSION, search_query2, radius, LIMIT)
url2

'https://api.foursquare.com/v2/venues/search?client_id=WCK535NG04YZTNLZP3JNTI215B45UCMROLCZAO2DCWL4ZUQ2&client_secret=1UVSHMRNYHQBMSEOGAL4GP3CACE24GL3LQ40DFVWADWGOY3Q&ll=4.59808,-74.0760439&v=20180604&query=Joyeria&radius=1000&limit=30'

#### Now let's examine the results in Bogota

In [23]:
resultsB = requests.get(url2).json()
resultsB

{'meta': {'code': 200, 'requestId': '5d5ea620c53093002c578cde'},
 'response': {'venues': [{'id': '4ee6156b9adf398201023461',
    'name': 'Joyería Citizen',
    'location': {'lat': 4.598725789365495,
     'lng': -74.07381194311554,
     'labeledLatLngs': [{'label': 'display',
       'lat': 4.598725789365495,
       'lng': -74.07381194311554}],
     'distance': 257,
     'cc': 'CO',
     'country': 'Colombia',
     'formattedAddress': ['Colombia']},
    'categories': [{'id': '4bf58dd8d48988d111951735',
      'name': 'Jewelry Store',
      'pluralName': 'Jewelry Stores',
      'shortName': 'Jewelry',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/jewelry_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1566484000',
    'hasPerk': False},
   {'id': '50c3a0ace4b047e9d431fb3b',
    'name': 'Joyeria Centro Seiko',
    'location': {'address': 'Cra 15 calle 91',
     'lat': 4.599064,
     'lng': -74.073684,
     'labeledLatLngs': [{'label': 'dis

In [24]:
venues2 = resultsB['response']['venues']
dataframe = json_normalize(venues2)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d111951735', 'name': 'Jewelry Store', 'pluralName': 'Jewelry Stores', 'shortName': 'Jewelry', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/jewelry_', 'suffix': '.png'}, 'primary': True}]",False,4ee6156b9adf398201023461,NaN,CO,NaN,Colombia,NaN,257,[Colombia],"[{'label': 'display', 'lat': 4.598725789365495, 'lng': -74.07381194311554}]",4.598726,-74.073812,NaN,NaN,Joyería Citizen,v-1566484000
1,"[{'id': '4bf58dd8d48988d111951735', 'name': 'Jewelry Store', 'pluralName': 'Jewelry Stores', 'shortName': 'Jewelry', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/jewelry_', 'suffix': '.png'}, 'primary': True}]",False,50c3a0ace4b047e9d431fb3b,Cra 15 calle 91,CO,NaN,Colombia,NaN,283,"[Cra 15 calle 91, Colombia]","[{'label': 'display', 'lat': 4.599064, 'lng': -74.073684}]",4.599064,-74.073684,NaN,NaN,Joyeria Centro Seiko,v-1566484000
2,"[{'id': '4bf58dd8d48988d1ff941735', 'name': 'Miscellaneous Shop', 'pluralName': 'Miscellaneous Shops', 'shortName': 'Shop', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/default_', 'suffix': '.png'}, 'primary': True}]",False,4f4e5c004fc64d33baa0b149,Calle 12b 6 58,CO,"Distrito Capital, Bogotá",Colombia,NaN,291,"[Calle 12b 6 58, Distrito Capital, Bogotá, Colombia]","[{'label': 'display', 'lat': 4.5996960724789, 'lng': -74.07397627830505}]",4.599696,-74.073976,051,NaN,Joyería Nivada,v-1566484000
3,"[{'id': '4bf58dd8d48988d111951735', 'name': 'Jewelry Store', 'pluralName': 'Jewelry Stores', 'shortName': 'Jewelry', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/jewelry_', 'suffix': '.png'}, 'primary': True}]",False,55fb03e1498e3d56f082a435,Calle 12b No. 6-57,CO,Candelaria,Colombia,Centro,301,"[Calle 12b No. 6-57 (Centro), Candelaria, Bogotá D.C., Colombia]","[{'label': 'display', 'lat': 4.599928171829357, 'lng': -74.07405860862978}]",4.599928,-74.074059,NaN,Bogotá D.C.,Joyería Roma Relojería,v-1566484000
4,"[{'id': '4bf58dd8d48988d111951735', 'name': 'Jewelry Store', 'pluralName': 'Jewelry Stores', 'shortName': 'Jewelry', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/jewelry_', 'suffix': '.png'}, 'primary': True}]",False,52597395498e80ed5246b635,Calle 13 # 6 - 03,CO,NaN,Colombia,NaN,371,"[Calle 13 # 6 - 03, Colombia]","[{'label': 'display', 'lat': 4.599854469299316, 'lng': -74.07321166992188}]",4.599854,-74.073212,NaN,NaN,Joyería Maria Hel,v-1566484000


#### Let's define the data frame with the information that is of interest

In [25]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Joyería Citizen,Jewelry Store,NaN,CO,NaN,Colombia,NaN,257,[Colombia],"[{'label': 'display', 'lat': 4.598725789365495, 'lng': -74.07381194311554}]",4.598726,-74.073812,NaN,NaN,4ee6156b9adf398201023461
1,Joyeria Centro Seiko,Jewelry Store,Cra 15 calle 91,CO,NaN,Colombia,NaN,283,"[Cra 15 calle 91, Colombia]","[{'label': 'display', 'lat': 4.599064, 'lng': -74.073684}]",4.599064,-74.073684,NaN,NaN,50c3a0ace4b047e9d431fb3b
2,Joyería Nivada,Miscellaneous Shop,Calle 12b 6 58,CO,"Distrito Capital, Bogotá",Colombia,NaN,291,"[Calle 12b 6 58, Distrito Capital, Bogotá, Colombia]","[{'label': 'display', 'lat': 4.5996960724789, 'lng': -74.07397627830505}]",4.599696,-74.073976,051,NaN,4f4e5c004fc64d33baa0b149
3,Joyería Roma Relojería,Jewelry Store,Calle 12b No. 6-57,CO,Candelaria,Colombia,Centro,301,"[Calle 12b No. 6-57 (Centro), Candelaria, Bogotá D.C., Colombia]","[{'label': 'display', 'lat': 4.599928171829357, 'lng': -74.07405860862978}]",4.599928,-74.074059,NaN,Bogotá D.C.,55fb03e1498e3d56f082a435
4,Joyería Maria Hel,Jewelry Store,Calle 13 # 6 - 03,CO,NaN,Colombia,NaN,371,"[Calle 13 # 6 - 03, Colombia]","[{'label': 'display', 'lat': 4.599854469299316, 'lng': -74.07321166992188}]",4.599854,-74.073212,NaN,NaN,52597395498e80ed5246b635
5,Joyeria Candelaria,Jewelry Store,NaN,CO,NaN,Colombia,NaN,384,[Colombia],"[{'label': 'display', 'lat': 4.601404, 'lng': -74.075108}]",4.601404,-74.075108,NaN,NaN,50ba3782e4b05d0a8e3254eb
6,Arte joyeria,Jewelry Store,Carrera 6 No.12-25,CO,NaN,Colombia,NaN,273,"[Carrera 6 No.12-25, Colombia]","[{'label': 'display', 'lat': 4.598859, 'lng': -74.073704}]",4.598859,-74.073704,NaN,NaN,5307d36d498e227868d2f1de
7,Joyeria Continental,Jewelry Store,NaN,CO,NaN,Colombia,NaN,931,[Colombia],"[{'label': 'display', 'lat': 4.605955600738525, 'lng': -74.07321166992188}]",4.605956,-74.073212,NaN,NaN,5187f632498ef92261ed36bc
8,Vanesa Solís Joyería,Jewelry Store,NaN,CO,Bogotá,Colombia,NaN,666,"[Bogotá, Bogotá D.C., Colombia]","[{'label': 'display', 'lat': 4.6020966, 'lng': -74.071594}]",4.602097,-74.071594,110321,Bogotá D.C.,5a88b76d51950e7c1458e857
9,mido relojeria y joyeria,Jewelry Store,NaN,CO,NaN,Colombia,NaN,719,[Colombia],"[{'label': 'display', 'lat': 4.60373664772403, 'lng': -74.07290558799528}]",4.603737,-74.072906,NaN,NaN,4fb3f411e4b01462acbfc7f9


##### Now let's sum up the name of the jewelries close to our point of interest

In [26]:
dataframe_filtered.name

0    Joyería Citizen         
1    Joyeria Centro Seiko    
2    Joyería Nivada          
3    Joyería Roma Relojería  
4    Joyería Maria Hel       
5    Joyeria Candelaria      
6    Arte joyeria            
7    Joyeria Continental     
8    Vanesa Solís Joyería    
9    mido relojeria y joyeria
Name: name, dtype: object

In [27]:
# lets take a Look at the location in the city

In [28]:
venues_maps = folium.Map(location=[latitude2, longitude2], zoom_start=13) 

# add a green circle marker to represent the Floripa Jewelry
folium.features.CircleMarker(
    [latitude2, longitude2],
    radius=10,
    color='green',
    popup='Joyería Maria Hel',
    fill = True,
    fill_color = 'green',
    fill_opacity = 0.5
).add_to(venues_maps)

# add the jewelries as purplee circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='purple',
        popup=label,
        fill = True,
        fill_color='purple',
        fill_opacity=0.5
    ).add_to(venues_maps)
venues_maps

####  Let's explore in detail some of the jewelries.

In [29]:
venue_id = '4ee6156b9adf398201023461' # Joyeria Citizen
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4ee6156b9adf398201023461?client_id=WCK535NG04YZTNLZP3JNTI215B45UCMROLCZAO2DCWL4ZUQ2&client_secret=1UVSHMRNYHQBMSEOGAL4GP3CACE24GL3LQ40DFVWADWGOY3Q&v=20180604'

In [30]:
result3 = requests.get(url).json()
print(result3['response']['venue'].keys())
result3['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'likes', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto'])


{'id': '4ee6156b9adf398201023461',
 'name': 'Joyería Citizen',
 'contact': {},
 'location': {'lat': 4.598725789365495,
  'lng': -74.07381194311554,
  'labeledLatLngs': [{'label': 'display',
    'lat': 4.598725789365495,
    'lng': -74.07381194311554}],
  'cc': 'CO',
  'country': 'Colombia',
  'formattedAddress': ['Colombia']},
 'canonicalUrl': 'https://foursquare.com/v/joyer%C3%ADa-citizen/4ee6156b9adf398201023461',
 'categories': [{'id': '4bf58dd8d48988d111951735',
   'name': 'Jewelry Store',
   'pluralName': 'Jewelry Stores',
   'shortName': 'Jewelry',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/jewelry_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 1},
 'likes': {'count': 1,
  'groups': [{'type': 'others',
    'count': 1,
    'items': [{'id': '17885492',
      'firstName': 'Zoniferland',
      'gender': 'female',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/17885492-FWVTUGAQZTKFC

In [31]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has no ratinmgs.')

9.2


##### Really good rating, lets check another Jewelry.

In [32]:
venue_id = '4bb8e2367421a593c439c240' # ID of Joyeria Andino
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.8


##### Let's analyze this one.

In [33]:
result['response']['venue']['tips']['count']

312

#### Let's get the venues tips
##### Again, the idea would be to get all the tips but as we have a personal account it would not allow us...

In [34]:
limit = 2
url3 = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results4 = requests.get(url).json()
results4

{'meta': {'code': 200, 'requestId': '5d5ea636a197930038cc5de9'},
 'response': {'venue': {'id': '4bb8e2367421a593c439c240',
   'name': 'Centro Comercial Andino',
   'contact': {'phone': '+5716213111',
    'formattedPhone': '+57 1 6213111',
    'twitter': 'centro_andino',
    'instagram': 'centrocomercialandino',
    'facebook': '37192939013',
    'facebookUsername': 'centrocomercial.andino',
    'facebookName': 'CENTRO COMERCIAL ANDINO'},
   'location': {'address': 'Carrera 11 # 82 - 71',
    'lat': 4.666714547946073,
    'lng': -74.05309177776574,
    'labeledLatLngs': [{'label': 'display',
      'lat': 4.666714547946073,
      'lng': -74.05309177776574}],
    'postalCode': '110221',
    'cc': 'CO',
    'city': 'Bogotá',
    'state': 'Bogotá D.C.',
    'country': 'Colombia',
    'formattedAddress': ['Carrera 11 # 82 - 71',
     'Bogotá, Bogotá D.C.',
     'Colombia']},
   'canonicalUrl': 'https://foursquare.com/v/centro-comercial-andino/4bb8e2367421a593c439c240',
   'categories': [{'id

In [40]:
tips2 = results4['response']['venue']

tip2 = results4['response']['venue']
tip2.keys()

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'beenHere', 'specials', 'photos', 'reasons', 'description', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])